Make the changes as per data dimensions

# Data

In [ ]:
import numpy as np

In [ ]:
"""
Number of data points: 300
Number of classes: 20
"""

# INPUTS
W = np.ones((300, 200))
X = np.ones((300, 200))

# OUTPUT
Y = np.random.rand(300, 20)

In [ ]:
# Train test split
W_train, X_train, Y_train = W[:200], X[:200], Y[:200]
W_test, X_test, Y_test = W[200:], X[200:], Y[200:]

# Model Definition

In [ ]:
from spektral.layers import GraphSageConv
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, Input, Concatenate
from tensorflow.keras.models import Model

In [ ]:
batch_size = 32
graph_embedding_size = 300

In [ ]:
def myModel():    
    inputs_W = Input(shape=(200), name='W') # [32 X 200]
    inputs_X = Input(shape=(200), name='X') # [32 X 200]
    
    inputs_sparse = Input(shape=(2*batch_size,), name='sparse_matrix', sparse=True, dtype=tf.float64)  # [64 X 64] 
    
    # GraphSage
    inpEmbedding = Concatenate(axis=0)([inputs_W, inputs_X]) # [64 X 200]
    outEmbedding = GraphSageConv(channels=graph_embedding_size)([inpEmbedding, inputs_sparse]) # [64 X 300]
    
    W_modified = outEmbedding[0:batch_size] # [32 X 300]
    X_modified = outEmbedding[batch_size:2*batch_size] # [32 X 300]

    # Final Layers
    f = Concatenate(axis=1)([W_modified, X_modified]) # [32 X 600]
    dropout = Dropout(0.5)(f)
    Softmax = Dense(20, activation="softmax")(dropout)
    model = Model(inputs=[
        inputs_W,
        inputs_X,
        inputs_sparse,
    ], outputs=Softmax)

    model.compile(optimizer='adam', loss='categorical_crossentropy')
    return model

model = myModel()

print("Inputs:",[k.shape for k in model.inputs])
print("Outputs:", [k.shape for k in model.outputs], "\n")

print(model.summary())

del model

# Data Generator

In [ ]:
def cos_sim(A, B):
    return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

def getIndicesAndValues(batch_W, batch_X):
    indices = []
    values = []
    
    """
    In the constructed graph
    
    [0:batch_size] => W
    [batch_size:] => X
    
    Ex:
    Batch Size: 32
    
    0: Img1
    1: Img2
    .
    .
    
    31: Img32
    32: Text1
    33: Text2
    .
    .
    63: Text32
    
    """
    
    # W <-> W edges
    for i in range(batch_size):
        for j in range(batch_size):
            # Some Criterion
            sim_score = cos_sim(batch_W[i],batch_W[j])
            if sim_score >= 0.5:
                indices.append([i,j])
                values.append(sim_score)
    
    # X <-> X edges
    for i in range(batch_size):
        for j in range(batch_size):
            # Some Criterion
            sim_score = cos_sim(batch_X[i],batch_X[j])
            if sim_score >= 0.5:
                indices.append([i+batch_size,j+batch_size])
                values.append(sim_score)
    
    """
    # For W <-> X edges, indices would be of the form [i, j+batch_size]
    for i in range(batch_size):
        for j in range(batch_size):
            # Some Criterion
            if criterion(batch_X[i],batch_X[j]) is True:
                indices.append([i,j+batch_size])
                values.append(edge_weight)
    """

    indices = np.array(indices)
    values = np.array(values)

    return indices, values

In [ ]:
def datagenerator(W, X, Y):
    i = 0
    while True:
        if i + batch_size <= len(W):
            
            inp_W = W[i:i+batch_size]
            inp_X = X[i:i+batch_size]
            
            # Get Indices and Values
            indices, values = getIndicesAndValues(inp_W, inp_X)
            
            # Construct Sparse Matrix for GraphSAGE
            inp_sparse = tf.SparseTensor(
                            indices = indices,
                            values = values,
                            dense_shape = (batch_size*2,batch_size*2)
                        )
            
            out_Y = Y[i:i+batch_size]

            yield [inp_W, inp_X, inp_sparse], out_Y
            i+=batch_size
        else:
            i= 0
            continue
            
x = next(datagenerator(W, X, Y))

print("Inputs:")
for a in x[0]:
    print(a.shape)
    
print("\nOutput:")
print(x[1].shape)

del x

# Model Training

In [ ]:
model = myModel()

num_epochs = 5

for e in range(num_epochs):
    print("Epoch:",e+1)
    
    history = model.fit(x=datagenerator(W_train, X_train, Y_train),
                        steps_per_epoch=int(len(W_train) / batch_size),
                        epochs=1,
                        verbose=1
                        )
    
    y_pred = model.predict(x = datagenerator(W_test, X_test, Y_test),
                          steps=int(len(W_test) / batch_size)
                          )
    
    # Code For evaluation....